**DATA**
<br>
src: https://snap.stanford.edu/data/twitter_combined.txt.gz
<br>
content: Directed edges

Download data

In [ ]:
import requests

url = 'https://snap.stanford.edu/data/twitter_combined.txt.gz'
file_name = 'twitter_combined.txt.gz'

with requests.get(url, stream=True) as r:
    with open(file_name, 'wb') as f:
        r.raise_for_status()
        for chunk in r.iter_content():
            f.write(chunk)

In [ ]:
!gzip -d twitter_combined.txt.gz

**Pyspark - computing page ranks**

Spark entry-point

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
        builder. \
        appName("PythonPageRank"). \
        getOrCreate()

sc = spark.sparkContext

Import method to compute page ranks

In [3]:
from page_rank import calculate_page_ranks

Compute page ranks

In [4]:
page_ranks = calculate_page_ranks(sc, 'twitter_combined.txt', 2, verbose=True)

Completed iteration: 0
Completed iteration: 1
Completed iteration: 2
Completed iteration: 3
Completed iteration: 4
Completed iteration: 5
Completed iteration: 6
Completed iteration: 7
Completed iteration: 8
Completed iteration: 9
Completed iteration: 10
Completed iteration: 11
Completed iteration: 12
Completed iteration: 13
Completed iteration: 14
Completed iteration: 15
Completed iteration: 16
Completed iteration: 17
Completed iteration: 18
Completed iteration: 19
Completed iteration: 20
Completed iteration: 21
Completed iteration: 22
Completed iteration: 23
Completed iteration: 24
Completed iteration: 25
Completed iteration: 26
Completed iteration: 27
Completed iteration: 28
Completed iteration: 29
Completed iteration: 30
Completed iteration: 31
Completed iteration: 32
Completed iteration: 33
Completed iteration: 34
Completed iteration: 35
Completed iteration: 36
Completed iteration: 37
Completed iteration: 38
Completed iteration: 39
Completed iteration: 40
Completed iteration: 41
Co

Sort pages by page ranks (descending order)

In [6]:
page_ranks_desc = page_ranks.sortBy(lambda page: page[1], ascending=False)

Top 5 pages

In [7]:
print(*page_ranks_desc.take(5), sep='\n')

('115485051', 0.009997326501781917)
('116485573', 0.00849979928955559)
('813286', 0.0020088641287148044)
('40981798', 0.001956607448113627)
('11348282', 0.001819841752132813)


Save page ranks

In [14]:
!rm -rf twitter_page_ranks

In [15]:
page_ranks_desc.map(lambda x: x[0] + ' ' + str(x[1])). \
                saveAsTextFile('twitter_page_ranks')